In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
import tensorflow as tf
import google.datalab.bigquery as dlbq

In [4]:
print(tf.__version__)
if tf.test.is_built_with_cuda():
    print ("Built with cuda")
if tf.test.is_gpu_available():
    print("GPU available.")

1.12.0


In [5]:
tf.test.is_built_with_cuda()

False

In [6]:
tf.logging.set_verbosity(tf.logging.INFO)

---
##### From Jupyter notebook to python package
From exploration to production.

This little tool dumps a given function to a file with the same name in a certain package

In [7]:
PACKAGE="./train"
from tools import make_src_dumper
write_py = make_src_dumper(PACKAGE)

---

# Training and Evaluation Data
Training and evaluation data should be provided in files already.

If not, please go back an run ```Processing_ATL_JUNE.ipynb```

#### Fetch a sample file for examination

In [8]:
DATASET='atl_june'
a_training_file = !gsutil ls gs://going-tfx/$DATASET/train_data/atl_june_csv-00000-of-*
a_training_file = a_training_file[0]
TEMP_DIR='/tmp/atl_june/{}'.format(DATASET)
!rm -rf $TEMP_DIR
!mkdir -p $TEMP_DIR
!gsutil cp $a_training_file $TEMP_DIR
a_training_file = !ls $TEMP_DIR
a_training_file = os.path.join(TEMP_DIR,a_training_file[0])
res=!wc -l $a_training_file
res=res[0].split(" ")
print()
print("{} records in {}".format(res[0], res[1]))

Copying gs://going-tfx/atl_june/train_data/atl_june_csv-00000-of-00024...
/ [1 files][143.0 KiB/143.0 KiB]                                                
Operation completed over 1 objects/143.0 KiB.                                    

1000 records in /tmp/atl_june/atl_june/atl_june_csv-00000-of-00024


#### Have a look into the first training data file

This data is at the **training data** stage. It's got all and only the columns we want. Is has been normalized and integerized. We'll use the ```tf.feature_column``` API to further prepare categorical features.

In [9]:
from train.model_config import ORDERED_TRAINING_COLUMNS
probe = pd.read_csv(a_training_file, names=ORDERED_TRAINING_COLUMNS)
probe.sample(frac=1.0)[:10]

,AIRLINE,ARR,ARR_DELAY,ARR_LAT,ARR_LON,DEP_DELAY,DEP_DOW,DEP_HOD,DEP_LAT,DEP_LON,DIFF_LAT,DIFF_LON,DISTANCE,MEAN_TEMP_ARR,MEAN_TEMP_DEP,MEAN_VIS_ARR,MEAN_VIS_DEP,WND_SPD_ARR,WND_SPD_DEP
97,0,12,1.0,42.36,-71.00,0.076923,5,13,33.63,-84.42,0.567288,0.933419,0.196172,0.315884,0.613445,0.483871,0.671875,0.007201,0.263566
693,1,106,39.0,31.53,-84.19,0.127159,2,8,33.63,-84.42,0.318150,0.791774,0.015087,0.604693,0.634454,0.403226,1.000000,0.008301,0.542636
777,0,6,-21.0,26.07,-80.15,0.078493,1,8,33.63,-84.42,0.192547,0.835159,0.113756,0.678700,0.634454,0.489247,0.984375,0.003800,0.534884
268,3,7,8.0,25.79,-80.29,0.103611,5,10,33.63,-84.42,0.186105,0.833655,0.116937,0.563177,0.613445,0.489247,0.671875,0.004800,0.263566
155,1,99,12.0,38.03,-87.53,0.125589,2,21,33.63,-84.42,0.467679,0.755906,0.061418,0.519856,0.613445,0.489247,0.953125,0.002700,0.294574
960,0,107,30.0,33.67,-117.86,0.111460,2,18,33.63,-84.42,0.367380,0.430198,0.415640,0.398917,0.634454,0.295699,0.984375,0.004300,0.317829
983,0,49,2.0,35.81,-83.99,0.080063,2,20,33.63,-84.42,0.416609,0.793922,0.016644,0.462094,0.634454,0.370968,1.000000,0.006701,0.542636
772,6,3,-15.0,40.69,-74.16,0.073783,2,9,33.63,-84.42,0.528870,0.899484,0.150884,0.326715,0.634454,0.198925,1.000000,0.011401,0.542636
182,5,23,-24.0,29.64,-95.27,0.073783,2,9,33.63,-84.42,0.274672,0.672788,0.139405,0.671480,0.613445,0.483871,0.953125,0.007001,0.294574
890,1,70,206.0,33.67,-78.92,0.395604,2,17,33.63,-84.42,0.367380,0.848368,0.053715,0.548736,0.634454,0.397849,1.000000,0.009701,0.542636


In [10]:
probe.describe()

,AIRLINE,ARR,ARR_DELAY,ARR_LAT,ARR_LON,DEP_DELAY,DEP_DOW,DEP_HOD,DEP_LAT,DEP_LON,DIFF_LAT,DIFF_LON,DISTANCE,MEAN_TEMP_ARR,MEAN_TEMP_DEP,MEAN_VIS_ARR,MEAN_VIS_DEP,WND_SPD_ARR,WND_SPD_DEP
count,1000.000000,1000.000000,1000.00000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00,1000.00,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.224000,43.887000,11.20800,35.64568,-86.491930,0.098691,3.571000,14.435000,33.63,-84.42,0.412829,0.767054,0.128623,0.528343,0.597059,0.435871,0.850750,0.007501,0.390248
std,2.176364,36.665059,33.44885,5.36921,12.619807,0.046621,1.703485,4.633276,0.00,0.00,0.123515,0.135522,0.115086,0.126634,0.105001,0.074737,0.160310,0.031538,0.134370
min,0.000000,0.000000,-30.00000,18.33000,-157.920000,0.047096,1.000000,5.000000,33.63,-84.42,0.014493,0.000000,0.000000,0.063177,0.445378,0.010753,0.484375,0.001000,0.062015
25%,0.000000,13.000000,-7.00000,30.69000,-89.970000,0.076923,2.000000,10.000000,33.63,-84.42,0.298827,0.729704,0.062598,0.450812,0.466386,0.408602,0.687500,0.004500,0.294574
50%,1.000000,36.000000,1.00000,35.87000,-82.530000,0.080063,4.000000,14.000000,33.63,-84.42,0.417989,0.809601,0.105027,0.554152,0.613445,0.473118,0.953125,0.006101,0.426357
75%,2.000000,68.000000,18.00000,40.49000,-78.920000,0.102433,4.000000,19.000000,33.63,-84.42,0.524270,0.848368,0.150528,0.615523,0.634454,0.489247,0.984375,0.008201,0.519380
max,16.000000,169.000000,292.00000,47.45000,-64.970000,0.500785,7.000000,23.000000,33.63,-84.42,0.684380,0.998174,1.000000,1.000000,0.970588,0.489247,1.000000,1.000000,0.542636


---
# Feature engineering for categorical columns

Categorical columns need to be treated once more to derive at numerical input suitable for model training. That involves bucketizing, the use of dictionaries, feature crossing and embedding

#### Find ranges to bucketize latitude and longitude 
We can easily understand the range of values with the help of a bq query and ```pandas.describe()```

In [11]:
query="""
select 
    distinct arrival_airport as airport, arrival_lat as lat, arrival_lon as lon 
from 
    `bigquery-samples.airline_ontime_data.flights`
"""
locations = dlbq.Query(query).execute().result().to_dataframe()
locations.describe()

,lat,lon
count,344.000000,344.000000
mean,38.491570,-98.531599
std,8.547964,21.746974
min,13.480000,-176.640000
25%,33.450000,-111.675000
50%,38.715000,-93.300000
75%,42.907500,-82.497500
max,71.280000,-64.800000


In [12]:
lat_boundaries = range(10,80,5)
lat_boundaries

[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75]

In [13]:
lon_boundaries = range(-100, -55, 5)
lon_boundaries

[-100, -95, -90, -85, -80, -75, -70, -65, -60]

We're going to use those boundaries in the function below

#### Using tf feature_column api for bucketizing, crossing and embedding

In [14]:
def create_feature_columns():
    """
        returns: a dict of features columns for wide and deep input
    """
    
    from tensorflow.feature_column import indicator_column as ind
    from tensorflow.feature_column import numeric_column as num
    from tensorflow.feature_column import bucketized_column as buck
    from tensorflow.feature_column import crossed_column as cross
    from tensorflow.feature_column import embedding_column as emb
    from tensorflow.feature_column import categorical_column_with_identity as cid
    
    ################################################################
    #  Numerical columns for the pre-processed features
    ################################################################
    feature_columns = [
        num(col) for col in [
            'DEP_DELAY',  
            'MEAN_TEMP_DEP','MEAN_VIS_DEP','WND_SPD_DEP',
            'MEAN_TEMP_ARR','MEAN_VIS_ARR','WND_SPD_ARR',
            'DIFF_LAT','DIFF_LON','DISTANCE']]
    
    ################################################################
    #  categorical from ints, bucket counts from examination of the 
    #  full dataset
    ################################################################
    airline = ind(cid('AIRLINE', num_buckets=30))
    arrival = ind(cid('ARR', num_buckets=400))
    
    ################################################################
    #  Crossed and embedded
    ################################################################
    lat_boundaries = range(10,80,5)
    lon_boundaries = range(-100, -55, 5)
    cross_size = len(lat_boundaries) * len(lon_boundaries)

    arr_geo_emb = emb(cross([
        buck(num('ARR_LAT'), lat_boundaries), 
        buck(num('ARR_LON'), lon_boundaries)], cross_size), 10)

    dep_geo_emb = emb(cross([
        buck(num("DEP_LAT"), lat_boundaries), 
        buck(num("DEP_LON"), lon_boundaries)], cross_size), 10)

    dep_how_emb = emb(cross([
        cid("DEP_HOD", num_buckets=24), 
        cid("DEP_DOW", num_buckets=8)], 7*24), 10)

    ################################################################
    #  all together
    ################################################################
    return {
        'deep': feature_columns + [dep_how_emb, arr_geo_emb, dep_geo_emb],
        'wide': [airline, arrival]}
    
write_py(create_feature_columns)

'create_feature_columns written to ./train/create_feature_columns.py.'

In [15]:
create_feature_columns()

{'deep': [_NumericColumn(key='DEP_DELAY', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
  _NumericColumn(key='MEAN_TEMP_DEP', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
  _NumericColumn(key='MEAN_VIS_DEP', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
  _NumericColumn(key='WND_SPD_DEP', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
  _NumericColumn(key='MEAN_TEMP_ARR', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
  _NumericColumn(key='MEAN_VIS_ARR', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
  _NumericColumn(key='WND_SPD_ARR', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
  _NumericColumn(key='DIFF_LAT', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
  _NumericColumn(key='DIFF_LON', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
  _NumericColumn(key='DISTANCE'

These feature columns encode a construction plan. The ```tf.feature_column.input_column()``` helper will construct a sub-graph from this plan and feed the root (the *result*) of the graph into the model. You see the pattern: All parts of the tensor graph are created within the session/graph context of the ```Estimator``` API. Never outside of it.

# The available hypotheses
Please see [04_Hypotheses.ipynb](04_Hypotheses.ipynb) for more insight into the various hypotheses functions

In [16]:
from train.make_hypotheses import make_hypotheses
all_hypotheses = make_hypotheses()
import inspect
print(inspect.getsource(all_hypotheses['linear']))

def hypothesis_linear(features, feature_columns, options):
    
    import tensorflow as tf
    from train.train_tools import weight_summary

    with tf.name_scope('Linear'):
    
        all_feature_columns = feature_columns['wide'] + feature_columns['deep']

        input_layer = tf.feature_column.input_layer( 
            features, feature_columns=all_feature_columns)

        out = tf.layers.dense(input_layer, 1, activation=None)
        weight_summary(out)
    
    return out



# The model function
The model function is responsible for providing different variants of the actual model suitable for training, evaluation and prediction

In [17]:
def make_model_fn(feature_columns, options, hypothesis):
    
    import tensorflow as tf
    from train.make_hypotheses import make_hypotheses
    
    optimizers={
        "sgd": tf.train.GradientDescentOptimizer(learning_rate=options['learning_rate']),
        "adam": tf.train.AdamOptimizer(learning_rate=options['learning_rate']),
        "adagrad": tf.train.AdagradOptimizer(learning_rate=options['learning_rate'])
    }
    
    def _model_fn(features, labels, mode):

        out = hypothesis(features, feature_columns, options)

        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode, predictions=out)


        labels = tf.expand_dims(labels, -1)
        loss = tf.losses.mean_squared_error(labels, out)
        mean_error=tf.metrics.mean(tf.abs(labels-out))

        if mode == tf.estimator.ModeKeys.EVAL:    
            return tf.estimator.EstimatorSpec(
                mode=mode,
                loss = loss,
                eval_metric_ops={'mean_error': mean_error}
            )

        else:
            optimizer = optimizers[options['optimizer']]
            train_op = optimizer.minimize(loss, global_step=tf.train.get_or_create_global_step())

            grads = optimizer.compute_gradients(loss)
            for g in grads:
                name = "%s-grad" % g[1].name
                name = name.replace(":", "_")
                tf.summary.histogram(name, g[0])
            
            return tf.estimator.EstimatorSpec(  
                mode,
                loss = loss,
                train_op = train_op)
        
    return _model_fn

write_py(make_model_fn)

'make_model_fn written to ./train/make_model_fn.py.'

---
# Available input functions
Please examine **```Input_Functions.ipynb```** for more information about the available input functions.

For now, have a look at the tfrecord input function we're going to use.

In [18]:
from train.make_input_fns import make_input_fns
tfr_input_fn = make_input_fns()['tfr']
import inspect
print(inspect.getsource(tfr_input_fn))

def make_tfr_input_fn(filename_pattern, batch_size, options):
    
    import tensorflow as tf
    from train.model_config import LABEL_COLUMN
    from train.model_config import TRAINING_METADATA

    feature_spec = TRAINING_METADATA.schema.as_feature_spec()

    def _input_fn():
        dataset = tf.data.experimental.make_batched_features_dataset(
            file_pattern=filename_pattern,
            batch_size=batch_size,
            features=feature_spec,
            shuffle_buffer_size=options['shuffle_buffer_size'],
            prefetch_buffer_size=options['prefetch_buffer_size'],
            reader_num_threads=options['reader_num_threads'],
            parser_num_threads=options['parser_num_threads'],
            sloppy_ordering=options['sloppy_ordering'],
            label_key=LABEL_COLUMN)

        if options['distribute']:
            return dataset 
        else:
            return dataset.make_one_shot_iterator().get_next()
    return _input_fn



---
#### Serving input function

In [19]:
def make_tft_serving_input_fn(metadata_dir):

    import tensorflow as tf
    import tensorflow_transform as tft
    from train.model_config import SIGNATURE_INT_COLUMNS
    from train.model_config import SIGNATURE_FLOAT_COLUMNS
    from train.model_config import SIGNATURE_STR_COLUMNS
        
    def _input_fn():
        # placeholders for all the raw inputs
        placeholders = {
            key: tf.placeholder(name = key, shape=[None], dtype=tf.int64)
            for key in SIGNATURE_INT_COLUMNS
        }
        placeholders.update({
            key: tf.placeholder(name = key, shape=[None], dtype=tf.float32)
            for key in SIGNATURE_FLOAT_COLUMNS
        })

        placeholders.update({
            key: tf.placeholder(name = key, shape=[None], dtype=tf.string)
            for key in SIGNATURE_STR_COLUMNS
        })

        # transform using the saved model in transform_fn        
        transform_output = tft.TFTransformOutput(transform_output_dir=metadata_dir)
        features = transform_output.transform_raw_features(placeholders)
            
        return tf.estimator.export.ServingInputReceiver(features, placeholders)

    return _input_fn

write_py(make_tft_serving_input_fn)

'make_tft_serving_input_fn written to ./train/make_tft_serving_input_fn.py.'

---
# Training and Evaluation

In [20]:
def train_and_evaluate(options):

    import tensorflow as tf
    from tensorflow.estimator import RunConfig
    from tensorflow.contrib.distribute import MirroredStrategy
    import mlflow
    
    from train.make_model_fn import make_model_fn
    from train.make_tft_serving_input_fn import make_tft_serving_input_fn
    from train.create_feature_columns import create_feature_columns
    from train.make_tfr_input_fn import make_tfr_input_fn
    from train.make_hypotheses import make_hypotheses
    from train.make_input_fns import make_input_fns

    
    with mlflow.start_run():

        log_params = [
            'base_dir',
            'file_format',
            'train_batch_size',
            'max_train_steps',
            'reader_num_threads',
            'parser_num_threads',
            'prefetch_buffer_size'    
        ]
        
        for key in log_params:
            mlflow.log_param(key, options[key])

        ##################################################################
        #   Train and Eval Input Functions
        ##################################################################
        make_input_fn=make_input_fns()[options['file_format']]

        train_input_fn = make_input_fn(options['train_data_pattern'], 
                                       options['train_batch_size'],
                                       options)    

        eval_input_fn = make_input_fn(options['eval_data_pattern'], 
                                      options['eval_batch_size'],
                                      options)


        ##################################################################
        #   Create the hypothesis and the model_fn
        ##################################################################
        hypothesis = make_hypotheses()[options['hypothesis']]    
        feature_columns = create_feature_columns()
        model_fn = make_model_fn(feature_columns, options, hypothesis )


        ##################################################################
        #    Train and Eval Spec
        ##################################################################
        serving_input_fn = make_tft_serving_input_fn(options['metadata_dir'])
        exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)

        train_spec = tf.estimator.TrainSpec(
            input_fn=train_input_fn, 
            max_steps=options['max_train_steps'])

        eval_spec = tf.estimator.EvalSpec(
            input_fn=eval_input_fn, exporters=exporter,
            steps = options['eval_steps'],
            throttle_secs=options['throttle_secs'],
            start_delay_secs=0)


        ##################################################################
        #   Create and configure the estimator
        ##################################################################
        strategy = MirroredStrategy() if options['distribute'] else None
        config = RunConfig(model_dir=options['model_dir'],
                           save_summary_steps=options['save_summary_steps'],
                           train_distribute=strategy, 
                           save_checkpoints_steps=options['save_checkpoints_steps'],
                           log_step_count_steps=options['log_step_count_steps'])

        estimator = tf.estimator.Estimator(
                config=config,
                model_fn=model_fn)


        ##################################################################
        #   Finally, train and evaluate the model
        ##################################################################
        final_eval = tf.estimator.train_and_evaluate(
            estimator, 
            train_spec=train_spec, 
            eval_spec=eval_spec)
        
        if final_eval[0] is not None:
            mlflow.log_metric('loss', final_eval[0]['loss'])
            mlflow.log_metric('mean_error', final_eval[0]['mean_error'])

        return final_eval
        
    
write_py(train_and_evaluate)

'train_and_evaluate written to ./train/train_and_evaluate.py.'

#### Run from within the notebook kernel

In [23]:
from train.train_tools import join_paths

DATASET='atl_june'

args={}

# file locations
args['base_dir']='gs://going-tfx/{}'.format(DATASET)
args['metadata_dir']='metadata'
args['model_dir']='model/v0'
args['train_data_pattern']='train_data/atl_june_tfr*'
args['eval_data_pattern']='eval_data/atl_june_tfr*'
args['file_format']='tfr'

# train and eval parameters
args['train_batch_size']=500
args['eval_batch_size']=1024
args['max_train_steps']=40000
args['eval_steps']=10

# Execution parameters
args['reader_num_threads']=16
args['parser_num_threads']=16
args['prefetch_buffer_size']=10000
args['shuffle_buffer_size']=10000
args['save_checkpoints_steps']=5000
args['log_step_count_steps']=200
args['throttle_secs']=30
args['distribute']=False
args['sloppy_ordering']=True
args['save_summary_steps']=100

# Model parameters
args['learning_rate']=1e-3
args['hypothesis']='linear'
args['optimizer']='sgd'

args = join_paths(args)

model_dir = args['model_dir']
print("using directory %s to store the model." % model_dir)
print("Remove the directory if you want to start from scratch".format(model_dir))
_ = !gsutil -m rm -rf $model_dir

print("=====================================================================================================")
from tools import create_runpy
create_runpy("run_task.sh", args)
!cat ./run_task.sh
print("=====================================================================================================")
print()

# if you want to keep your jupyter notebook clean, rather use run_task.sh from a terminal 
#res = train_and_evaluate(args)

using directory gs://going-tfx/atl_june/model/v0 to store the model.
Remove the directory if you want to start from scratch
export PYTHONPATH=${PYTHONPATH}:${PWD}
python -m train.task \
  --eval_steps="10"  \
  --parser_num_threads="16"  \
  --eval_data_pattern="gs://going-tfx/atl_june/eval_data/atl_june_tfr*"  \
  --train_batch_size="500"  \
  --shuffle_buffer_size="10000"  \
  --eval_batch_size="1024"  \
  --sloppy_ordering="True"  \
  --reader_num_threads="16"  \
  --file_format="tfr"  \
  --log_step_count_steps="200"  \
  --model_dir="gs://going-tfx/atl_june/model/v0"  \
  --throttle_secs="30"  \
  --optimizer="sgd"  \
  --learning_rate="0.001"  \
  --hypothesis="linear"  \
  --save_summary_steps="100"  \
  --max_train_steps="40000"  \
  --prefetch_buffer_size="10000"  \
  --metadata_dir="gs://going-tfx/atl_june/metadata"  \
  --train_data_pattern="gs://going-tfx/atl_june/train_data/atl_june_tfr*"  \
  --save_checkpoints_steps="5000"  \
  --base_dir="gs://going-tfx/atl_june"  \



Created the bash runner script named **```run_task.sh```**. 

Execute

``` 
bash run_task.sh

``` 

to perform the training from a shell.

In [79]:
def create_local_trainer(filename, args):
    export = "export PYTHONPATH=${PYTHONPATH}:${PWD}\n"
    cmd = "gcloud ml-engine local train --module-name train.task --package-path . -- \\\n"
    _args = "".join(['  --{}="{}"  \\\n'.format(key, value) for key, value in args.items() if value != False])
    with open(filename, 'w') as f:
        f.write(export+cmd+_args)
    
def create_submit_job(filename, args):
    _args = "".join(['    --{}="{}"  \\\n'.format(key, value) for key, value in args.items() if value != False])
    TFVERSION=1.8
    BUCKET='going-tfx'
    JOBDIR="gs://{}/jobs".format(BUCKET)
    PACKAGE="train"
    REGION="us-east1"
    cmd="""
    JOBNAME=homeintime_$(date -u +%y%m%d_%H%M%S)
    gcloud ml-engine jobs submit training $JOBNAME \\
    --region={} \\
    --module-name=train.task \\
    --package-path={} \\
    --job-dir={} \\
    --staging-bucket=gs://going-tfx \\
    --scale-tier=STANDARD_1 \\
    --runtime-version={} \\
    -- \\\n{}
    """.format(REGION, PACKAGE, JOBDIR, TFVERSION, _args)
    with open(filename, 'w') as f:
        f.write(cmd)
    return cmd

In [81]:
cmd = create_submit_job("submit_job.sh", args)
print(cmd)


    JOBNAME=homeintime_$(date -u +%y%m%d_%H%M%S)
    gcloud ml-engine jobs submit training $JOBNAME \
    --region=us-east1 \
    --module-name=train.task \
    --package-path=train \
    --job-dir=gs://going-tfx/jobs \
    --staging-bucket=gs://going-tfx \
    --scale-tier=STANDARD_1 \
    --runtime-version=1.8 \
    -- \
    --eval_steps="10"  \
    --parser_num_threads="16"  \
    --eval_data_pattern="gs://going-tfx/samples/eval_data/atl_june_tfr*"  \
    --train_batch_size="256"  \
    --shuffle_buffer_size="10000"  \
    --eval_batch_size="1024"  \
    --sloppy_ordering="True"  \
    --reader_num_threads="16"  \
    --file_format="tfr"  \
    --log_step_count_steps="200"  \
    --model_dir="gs://going-tfx/samples/model"  \
    --throttle_secs="30"  \
    --optimizer="sgd"  \
    --learning_rate="0.001"  \
    --hypothesis="linear"  \
    --save_summary_steps="100"  \
    --max_train_steps="8000"  \
    --prefetch_buffer_size="10000"  \
    --metadata_dir="gs://going-tfx/samples/m

In [82]:
!cat train_local.sh

export PYTHONPATH=${PYTHONPATH}:${PWD}
gcloud ml-engine local train --module-name train.task --package-path . -- \
  --eval_steps="10"  \
  --parser_num_threads="16"  \
  --eval_data_pattern="gs://going-tfx/samples/eval_data/atl_june_tfr*"  \
  --train_batch_size="256"  \
  --shuffle_buffer_size="10000"  \
  --eval_batch_size="1024"  \
  --sloppy_ordering="True"  \
  --reader_num_threads="16"  \
  --file_format="tfr"  \
  --log_step_count_steps="200"  \
  --model_dir="gs://going-tfx/samples/model"  \
  --throttle_secs="30"  \
  --optimizer="adam"  \
  --learning_rate="0.001"  \
  --hypothesis="linear"  \
  --save_summary_steps="100"  \
  --max_train_steps="8000"  \
  --prefetch_buffer_size="10000"  \
  --metadata_dir="gs://going-tfx/samples/metadata"  \
  --train_data_pattern="gs://going-tfx/samples/train_data/atl_june_tfr*"  \
  --save_checkpoints_steps="2000"  \
  --base_dir="gs://going-tfx/samples"  \
